In [111]:
# import lib
import fiftyone.zoo as foz
import fiftyone as fo
from fiftyone import ViewField as F
from keras.applications import VGG19
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam

In [147]:
# define samples count
subset_samples = 250

In [165]:
# Load a split of a zoo dataset
person_subset = foz.load_zoo_dataset(
    "open-images-v7",
    split="validation",
    label_types=["classifications"],
    classes=["Person"],
    max_samples=subset_samples,
    seed=51,
    shuffle=True,
    dataset_name="person_subset",
)

cat_subset = foz.load_zoo_dataset(
    "open-images-v7",
    split="validation",
    label_types=["classifications"],
    classes=["Cat"],
    max_samples=subset_samples,
    seed=51,
    shuffle=True,
    dataset_name="cat_subset",
)

dog_subset = foz.load_zoo_dataset(
    "open-images-v7",
    split="validation",
    label_types=["classifications"],
    classes=["Dog"],
    max_samples=subset_samples,
    seed=51,
    shuffle=True,
    dataset_name="dog_subset",
)

 100% |███████████████████| 250/250 [13.0s elapsed, 0s remaining, 23.0 files/s]      
Dataset info written to 'C:\Users\npaul\fiftyone\open-images-v7\info.json'
Loading existing dataset 'person_subset'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use
Found 2 images, downloading the remaining 248
 100% |███████████████████| 248/248 [12.0s elapsed, 0s remaining, 23.3 files/s]      
Dataset info written to 'C:\Users\npaul\fiftyone\open-images-v7\info.json'
Loading existing dataset 'cat_subset'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use
Found 5 images, downloading the remaining 245
 100% |███████████████████| 245/245 [12.0s elapsed, 0s remaining, 21.2 files/s]      
Dataset info written to 'C:\Users\npaul\fiftyone\open-images-v7\info.json'
Loading existing dataset 'dog_subset'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [166]:
# print out list of available datasets
print(fo.list_datasets())

['cat_dog_person_dataset', 'cat_subset', 'dog_subset', 'open-images-v7-train-500', 'open-images-v7-validation-25', 'open-images-v7-validation-500', 'person_subset']


In [169]:
# load dataset
_dataset = fo.load_dataset("person_subset")
dog_subset = fo.load_dataset("dog_subset")
cat_subset = fo.load_dataset("cat_subset")
# Merge the samples together into the same dataset
_dataset.merge_samples(dog_subset)
_dataset.merge_samples(cat_subset)
print(_dataset.view())
fo.pprint(_dataset.stats(include_media=True))

Dataset:     person_subset
Media type:  image
Num samples: 742
Sample fields:
    id:              fiftyone.core.fields.ObjectIdField
    filepath:        fiftyone.core.fields.StringField
    tags:            fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:        fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    positive_labels: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classifications)
    negative_labels: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classifications)
    detections:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
View stages:
    ---
{
    'samples_count': 742,
    'samples_bytes': 2190038,
    'samples_size': '2.1MB',
    'media_bytes': 235336086,
    'media_size': '224.4MB',
    'total_bytes': 237526124,
    'total_size': '226.5MB',
}


In [170]:
# Randomly shuffle the order of the samples in the dataset
_dataset = _dataset.shuffle()

In [171]:
sample_ids = _dataset.values("id")
# Split the sample IDs
train_ids, val_ids = train_test_split(sample_ids, test_size=0.3, random_state=42)
# Get the corresponding samples for training and testing
train_dataset = _dataset.select(train_ids)
val_dataset = _dataset.select(val_ids)
# Print the number of samples in each split
print("Number of training samples:", len(train_dataset))
print("Number of testing samples:", len(val_dataset))

Number of training samples: 519
Number of testing samples: 223


In [182]:
train_dataset

Dataset:     person_subset
Media type:  image
Num samples: 519
Sample fields:
    id:              fiftyone.core.fields.ObjectIdField
    filepath:        fiftyone.core.fields.StringField
    tags:            fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:        fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    positive_labels: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classifications)
    negative_labels: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classifications)
    detections:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
View stages:
    1. Shuffle(seed=None)
    2. Select(sample_ids=['6479cb4cfa920d5506fd3c68', '6479cb4cfa920d5506fd400e', '6479cb4cfa920d5506fd3c6c', ...], ordered=False)

In [173]:
# Define model parametes
num_classes = 3

In [174]:
# Define the model
model_scratch = Sequential()
model_scratch.add(VGG19(include_top=False, weights=None, input_shape=(224, 224, 3)))  # VGG19 without the top layer
model_scratch.add(Flatten())
model_scratch.add(Dense(256, activation='relu'))
model_scratch.add(Dense(num_classes, activation='softmax'))

In [177]:
# Compile the model
model_scratch.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
model_scratch.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

NameError: name 'X_train' is not defined